In [1]:
!nvidia-smi
!pip install -q ultralytics wandb

Sun May 21 15:50:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         On   | 00000000:00:1E.0 Off |                    0 |
|  0%   23C    P8    12W / 300W |      0MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: rishiraj. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ubuntu/.netrc


True

In [3]:
!curl -L "https://universe.roboflow.com/ds/SrOZL9LzLO?key=GemlNaWU3k" > traffic.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   886  100   886    0     0   1543      0 --:--:-- --:--:-- --:--:--  1543
100 1120M  100 1120M    0     0  64.9M      0  0:00:17  0:00:17 --:--:-- 67.7M 51.3M      0  0:00:21  0:00:07  0:00:14 64.7M


In [4]:
!unzip -q traffic.zip

In [5]:
import os
import shutil

# Set the path to the directory containing images and labels
data_dir = "export"

# Set the path to the directory where the train and validation sets will be created
train_dir = "train"
valid_dir = "valid"

# Set the ratio of data to be used for the validation set
validation_ratio = 0.2

# Get the list of all image files in the data directory
image_files = [f for f in os.listdir(os.path.join(data_dir, "images")) if f.endswith('.jpg')]

# Calculate the number of images to be used for validation
num_validation = int(validation_ratio * len(image_files))

# Shuffle the list of image files randomly
import random
random.seed(69)
random.shuffle(image_files)

# Split the image files into training and validation sets
train_image_files = image_files[num_validation:]
valid_image_files = image_files[:num_validation]

# Create the directories for the training and validation sets
os.makedirs(os.path.join(train_dir, "images"), exist_ok=True)
os.makedirs(os.path.join(train_dir, "labels"), exist_ok=True)
os.makedirs(os.path.join(valid_dir, "images"), exist_ok=True)
os.makedirs(os.path.join(valid_dir, "labels"), exist_ok=True)

# Copy the training set images and labels to the appropriate directories
for image_file in train_image_files:
    # Copy the image file
    shutil.copy(os.path.join(data_dir, "images", image_file), os.path.join(train_dir, "images"))
    # Copy the corresponding label file
    label_file = image_file.replace('.jpg', '.txt')
    shutil.copy(os.path.join(data_dir, "labels", label_file), os.path.join(train_dir, "labels"))

# Copy the validation set images and labels to the appropriate directories
for image_file in valid_image_files:
    # Copy the image file
    shutil.copy(os.path.join(data_dir, "images", image_file), os.path.join(valid_dir, "images"))
    # Copy the corresponding label file
    label_file = image_file.replace('.jpg', '.txt')
    shutil.copy(os.path.join(data_dir, "labels", label_file), os.path.join(valid_dir, "labels"))

In [6]:
file_path = 'data.yaml'
new_content = '''train: /home/ubuntu/oneAPI/train/images
val: /home/ubuntu/oneAPI/valid/images

nc: 11
names: ['biker', 'car', 'pedestrian', 'trafficLight', 'trafficLight-Green', 'trafficLight-GreenLeft', 'trafficLight-Red', 'trafficLight-RedLeft', 'trafficLight-Yellow', 'trafficLight-YellowLeft', 'truck']

roboflow:
  workspace: roboflow-gw7yv
  project: self-driving-car
  version: 3
  license: MIT
  url: https://universe.roboflow.com/roboflow-gw7yv/self-driving-car/dataset/3'''

with open(file_path, 'w') as file:
    file.write(new_content)

In [7]:
!sudo rm -rf export
!sudo rm -rf traffic.zip

In [8]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8m.pt")  # load a pretrained model (recommended for training)

# Use the model
model.train(data="data.yaml", epochs=50)  # train the model (just 50 for prototype)
metrics = model.val()  # evaluate model performance on the validation set

100%|██████████| 49.7M/49.7M [00:03<00:00, 16.6MB/s]
Ultralytics YOLOv8.0.105 🚀 Python-3.9.16 torch-2.0.1 CUDA:0 (NVIDIA A10G, 22548MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=data.yaml, epochs=50, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dyna

AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
100%|██████████| 6.23M/6.23M [00:01<00:00, 5.33MB/s]
AMP: checks passed ✅
optimizer: SGD(lr=0.01) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias
train: Scanning /home/ubuntu/oneAPI/train/labels... 23840 images, 2789 backgrounds, 0 corrupt: 100%|██████████| 23840/23840 [00:17<00:00, 1337.21it/s]
train: WARNING ⚠️ /home/ubuntu/oneAPI/train/images/1478021875081281646_jpg.rf.e9552980cf8c6fef4aa02cb84c6364f5.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/ubuntu/oneAPI/train/images/1478897760163798179_jpg.rf.5Pzrj3Eg3vZuyl7ztKAt.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/ubuntu/oneAPI/train/images/1478898145212453716_jpg.rf.6a92d7d7dd523160c990c4e4375bcea9.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/ubuntu/oneAPI/train/images/1478898145212453716_jpg.rf.nCaFkPk4AFMjTQAM4RTJ.jpg: 1 duplicate labels removed
train: New cache created: /home/ubuntu/oneAPI/train/labels